# Lab 4 - Career Conversation Agent (DeepSeek Refactored)

This is a refactored version of Lab 4 using DeepSeek with improvements in:
- **Cost efficiency**: DeepSeek is ~50% cheaper than OpenAI for outputs
- **Code organization**: Clean separation of concerns, proper function extraction
- **Maintainability**: Type hints, clear naming, modular design
- **KISS principle**: Simple, readable code without over-engineering

### Key Features:
1. AI-powered career assistant representing you professionally
2. Tool calling for recording user details and unknown questions
3. Real-time push notifications via Pushover
4. Web interface with Gradio
5. Ready for HuggingFace Spaces deployment

### Setup Required:
1. DeepSeek API key from https://platform.deepseek.com/
2. Pushover account from https://pushover.net/
3. Your LinkedIn PDF and summary in the `../me/` folder

## Setup Instructions

### 1. Get API Keys
- **DeepSeek**: Sign up at https://platform.deepseek.com/ and create an API key
- **Pushover**: 
  - Visit https://pushover.net/ and create a free account
  - Click "Create an Application/API Token" and name it (e.g., "Career Agent")
  - Install the Pushover app on your phone

### 2. Configure Environment Variables
Add these to your `.env` file:

```

DEEPSEEK_API_KEY=sk-...

PUSHOVER_USER=u...

PUSHOVER_TOKEN=a...

```

In [6]:
import json
import os
from typing import Dict, List, Any

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import requests

In [7]:
CONFIG = {
  "name": "Ed Donner",
  "model": "deepseek-chat",
  "linkedin_pdf": "../me/linkedin.pdf",
  "summary_file": "../me/summary.txt",
}

load_dotenv(override=True)

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

if not deepseek_api_key:
    raise ValueError("DEEPSEEK_API_KEY is not set in the environment variables")

deepseek = OpenAI(
    api_key=deepseek_api_key,
    base_url="https://api.deepseek.com/v1"
)

PUSHOVER_CONFIG = {
  "user": os.getenv("PUSHOVER_USER"),
  "token": os.getenv("PUSHOVER_TOKEN"),
  "url": "https://api.pushover.net/1/messages.json"
}

if not PUSHOVER_CONFIG["user"] or not PUSHOVER_CONFIG["token"]:
    raise ValueError("PUSHOVER_USER and PUSHOVER_TOKEN must be set in the environment variables")

print("✅ Configuration loaded:")
print(f"   Name: {CONFIG['name']}")
print(f"   Model: {CONFIG['model']}")
print(f"   DeepSeek API: {'✓' if os.getenv('DEEPSEEK_API_KEY') else '✗'}")
print(f"   Pushover: {'✓' if PUSHOVER_CONFIG['user'] and PUSHOVER_CONFIG['token'] else '✗'}")

✅ Configuration loaded:
   Name: Ed Donner
   Model: deepseek-chat
   DeepSeek API: ✓
   Pushover: ✓


## Notification Function

In [8]:
def send_notification(message: str) -> None:
  if not PUSHOVER_CONFIG["user"] or not PUSHOVER_CONFIG["token"]:
      print(f"⚠️  Pushover not configured. Would send: {message}")
      return

  try:
    payload = {
      "user": PUSHOVER_CONFIG["user"],
      "token": PUSHOVER_CONFIG["token"],
      "message": message,
      "title": "Career Agent Notification"
    }
    response = requests.post(PUSHOVER_CONFIG["url"], data=payload)
    response.raise_for_status()
    print(f"📱 Notification sent: {message}")    
  except Exception as e:
    print(f"❌ Notification failed: {e}")

## Tool functions

In [9]:
def record_user_details(email: str, name: str = "Name not provided", notes: str = "Not provided") -> Dict[str, str]:

  message = f"New contact: {name} ({email}\nNotes: {notes}"
  send_notification(message)
  return {"status": "recorded", "email": email}

def record_unknown_question(question: str) -> Dict[str, str]:
  message = f"Unknown question: {question}"
  send_notification(message)
  return {"status": "recorded", "question": question}

## Tool definitions

In [10]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "record_user_details",
            "description": "Record user contact information when they express interest in connecting",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "User's email address"
                    },
                    "name": {
                        "type": "string",
                        "description": "User's name (if provided)"
                    },
                    "notes": {
                        "type": "string",
                        "description": "Additional context about the conversation"
                    }
                },
                "required": ["email"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_unknown_question",
            "description": "Record questions that couldn't be answered for future improvement",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question that couldn't be answered"
                    }
                },
                "required": ["question"],
                "additionalProperties": False
            }
        }
    }
]

## Tool Execution Handler

In [11]:
def execute_tool_calls(tool_calls: List[Any]) -> List[Dict[str, Any]]:
  results = []
  for tool_call in tool_calls:
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    print(f"🔧 Executing tool: {tool_name}")

    tool_function = globals().get(tool_name)
    if tool_function:
      result = tool_function(**arguments)
    else:
      result = {"error": f"Unknown tool: {tool_name}"}

    results.append({
      "role": "tool",
      "content": json.dumps(result),
      "tool_call_id": tool_call.id
    })
    
  return results

## Profile loading

In [12]:
def load_profile() -> tuple[str, str]:

  try: 
    render = PdfReader(CONFIG["linkedin_pdf"])
    linkedin = "\n".join(
      page.extract_text()
      for page in render.pages
      if page.extract_text()
    )
  except FileNotFoundError:
    linkedin = "LinkedIn profile not found. Please add your LinkedIn PDF."
    print(f"⚠️  {linkedin}")


  try:
    with open(CONFIG["summary_file"], "r", encoding="utf-8") as file:
      summary = file.read()
  except FileNotFoundError:
    summary = "Summary not found. Please add your summary text file."
    print(f"⚠️  {summary}")

  return linkedin, summary


  # Load profile data
linkedin_profile, personal_summary = load_profile()
print(f"✅ Profile loaded: {len(linkedin_profile)} chars from LinkedIn, {len(personal_summary)} chars from summary")

✅ Profile loaded: 8260 chars from LinkedIn, 387 chars from summary


## System prompt builder

In [13]:
def build_system_prompt(name: str, summary: str, linkedin: str) -> str:
    """Build the system prompt for the AI agent."""
    return f"""You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience.

Your responsibility is to represent {name} for interactions on the website as faithfully as possible.

You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions.

Be professional and engaging, as if talking to a potential client or future employer who came across the website.

If you don't know the answer to any question, use your record_unknown_question tool to record it.

If the user is engaging in discussion, try to steer them towards getting in touch via email; \
ask for their email and record it using your record_user_details tool.

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

With this context, please chat with the user, always staying in character as {name}."""


# Build system prompt
SYSTEM_PROMPT = build_system_prompt(
    CONFIG["name"],
    personal_summary,
    linkedin_profile
)

## Main chat function

In [14]:
def chat(message: str, history: List[Dict[str, str]]) -> str:
  messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    *history,
    {"role": "user", "content": message}
  ]

  while True:
    response = deepseek.chat.completions.create(
      model=CONFIG["model"],
      messages=messages,
      tools=TOOLS
    )

    finish_reason = response.choices[0].finish_reason
    assistant_message = response.choices[0].message

    if finish_reason == "tool_calls":
      messages.append(assistant_message)
      tool_results = execute_tool_calls(assistant_message.tool_calls)
      messages.extend(tool_results)
    else:
      return assistant_message.content
    

## Launch Gradio Interface

In [ ]:
interface = gr.ChatInterface(
    fn=chat,
    type="messages",
    title=f"Chat with {CONFIG['name']}",
    description="Ask me about my career, experience, and skills!",
    examples=[
        "What's your background?",
        "What technologies do you work with?",
        "Can we connect? My email is example@email.com"
    ]
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


🔧 Executing tool: record_user_details
📱 Notification sent: New contact: Name not provided (test@example.com
Notes: User expressed interest in connecting after discussing technology stack
